In [4]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark


In [6]:
!pip3 install numpy

    100% |████████████████████████████████| 20.2MB 60kB/s  eta 0:00:01   13% |████▎                           | 2.7MB 11.7MB/s eta 0:00:02    19% |██████▏                         | 3.9MB 20.7MB/s eta 0:00:01    36% |███████████▉                    | 7.4MB 20.4MB/s eta 0:00:01    49% |███████████████▉                | 10.0MB 10.9MB/s eta 0:00:01    63% |████████████████████▎           | 12.8MB 26.3MB/s eta 0:00:01    73% |███████████████████████▍        | 14.8MB 11.6MB/s eta 0:00:01    82% |██████████████████████████▎     | 16.6MB 15.2MB/s eta 0:00:01    93% |██████████████████████████████  | 19.0MB 39.1MB/s eta 0:00:01


In [7]:
!pip3 install pandas

    100% |████████████████████████████████| 10.0MB 165kB/s eta 0:00:01   45% |██████████████▌                 | 4.5MB 11.5MB/s eta 0:00:01    85% |███████████████████████████▍    | 8.6MB 16.4MB/s eta 0:00:01    98% |███████████████████████████████▍| 9.9MB 35.7MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 2.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/03/27/e35e7c6e6a52fab9fcc64fc2b20c6b516eba930bb02b10ace3b38200d3ab/numpy-1.18.4-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 512kB 1.7MB/s eta 0:00:01


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
!ls

 dic487-587.zip  'Part1 (1).ipynb'   test.csv
 mapping.csv	  sample.csv	     train.csv


In [9]:
import numpy as np
import pandas as pd
from ast import literal_eval 
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import regexp_replace,col,array_contains,explode
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql.functions import concat_ws
from pyspark.ml.classification import LogisticRegression

In [10]:
pd_dataFrame = pd.read_csv(r'train.csv')
data_spark_dataFrame = spark.createDataFrame(pd_dataFrame)

pd_dataFrame['genre']= pd_dataFrame['genre'].apply(literal_eval)
all_genre = pd_dataFrame['genre'].to_list()


In [11]:
labels =['Drama','Comedy','Romance Film','Thriller','Action','World cinema','Crime Fiction','Horror','Black-and-white','Indie','Action/Adventure','Adventure','Family Film','Short Film','Romantic drama','Animation','Musical','Science Fiction','Mystery','Romantic comedy']
mat = np.zeros((len(pd_dataFrame),len(labels)))
for i,genre in enumerate(all_genre):
  for j,g in enumerate(genre):
    for k,label in enumerate(labels):
        if label==g:
          mat[i][k] = 1


In [13]:
labels = "Drama , Comedy , Romance Film , Thriller , Action , World cinema , Crime Fiction , Horror , Black-and-white , Indie , Action/Adventure , Adventure , Family Film , Short Film , Romantic drama , Animation , Musical , Science Fiction , Mystery , Romantic comedy"
np.savetxt("genre_lables.csv", mat, delimiter=",",fmt='%d',header=labels)


In [14]:
lables_dataFrame = pd.read_csv(r'genre_lables.csv')
test_pd_dataFrame = pd.read_csv(r'test.csv')


In [15]:
lables_spark_dataFrame = spark.createDataFrame(lables_dataFrame)
test_spark_dataFrame = spark.createDataFrame(test_pd_dataFrame)

In [16]:
ddataFrame1 = data_spark_dataFrame.withColumn("row_id", monotonically_increasing_id())
ddataFrame2 = lables_spark_dataFrame.withColumn("row_id", monotonically_increasing_id())
ddataFrame3 = test_spark_dataFrame.withColumn("row_id", monotonically_increasing_id())
dataFrame = ddataFrame1.join(ddataFrame2, "row_id").drop("row_id")
test_dataFrame = ddataFrame3.join(ddataFrame2, "row_id").drop("row_id")


In [17]:

regTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashingTF = HashingTF(inputCol="filtered", outputCol="features")
pipeline = Pipeline(stages=[regTokenizer, remover, hashingTF])



In [18]:
Mdl = pipeline.fit(dataFrame)
dataset = Mdl.transform(dataFrame)

Mdl2 = pipeline.fit(test_dataFrame)
test_dataset = Mdl2.transform(test_dataFrame)

In [19]:

dfList = []
labelColumns = lables_spark_dataFrame.columns
logReg = LogisticRegression(featuresCol = 'features',maxIter=500)


In [20]:
for labelColumn in labelColumns:
    logReg.setLabelCol(labelColumn)
    logRegModel = logReg.fit(dataset)
    pred = logRegModel.transform(test_dataset)
    pred = pred.withColumn("prediction",F.col("prediction").cast(IntegerType()))
    dfList.append(pred.select("movie_id","prediction"))

In [21]:
dataFrames_renamed = [dataFrame.selectExpr('movie_id', f'prediction as prediction_{i}') for i, dataFrame in enumerate(dfList)]
temp_dataFrame = reduce(lambda x, y: x.join(y, ['movie_id'], how='full'), dataFrames_renamed)
col_list = ['prediction_%d' % i for i in range(len(dfList))]
temp_dataFrame = temp_dataFrame.withColumn('predictions',concat_ws(" ",*col_list)).drop(*col_list).toPandas().to_csv("predictions_part1.csv",index=False)